In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import sys
import os.path
import urllib2

In [3]:
import htpar
reload(htpar)

<module 'htpar' from 'htpar/__init__.pyc'>

# NGINX Server

In [4]:
!mkdir -p NGINX/test-data
!chmod 777 NGINX/test-data
!cd NGINX && docker-compose -f docker-test.yml up -d
!sleep 3

chmod: changing permissions of 'NGINX/test-data': Operation not permitted
/work/htpar/NGINX
Creating nginx_htpar-test_1 ... 
Creating nginx_htpar-test_1
ting nginx_htpar-test_1 ... done

In [5]:
def read_url(url):
    return urllib2.urlopen(url).read()

def read_dir(url):
    import simplejson
    return simplejson.loads(read_url(url))
    
def write_url(url, data):
    import urllib2
    opener = urllib2.build_opener(urllib2.HTTPHandler)
    request = urllib2.Request(url, data=data)
    request.add_header('Content-Type', 'application/octet-stream')
    request.get_method = lambda: 'PUT'
    url = opener.open(request)
    return url

In [6]:
read_dir("http://localhost:28880")

[{'mtime': 'Thu, 28 Dec 2017 17:34:56 GMT',
  'name': '_test',
  'type': 'directory'},
 {'mtime': 'Thu, 28 Dec 2017 16:18:57 GMT',
  'name': 'cache',
  'type': 'directory'},
 {'mtime': 'Thu, 28 Dec 2017 16:41:11 GMT',
  'name': 'temp',
  'type': 'directory'},
 {'mtime': 'Thu, 28 Dec 2017 16:31:57 GMT',
  'name': 'test',
  'size': 9,
  'type': 'file'}]

In [7]:
test_data = "%g" % rand()
write_url("http://localhost:28880/_test/test", test_data)
result = read_url("http://localhost:28880/_test/test")
assert result == test_data
print result

0.407908


# tarfiles.py

In [8]:
with htpar.TarRecords("http://localhost:28880/_test/_test.tgz") as stream:
    for i in range(3):
        s = "%03d" % i
        stream.write(s, dict(s=s))

# curl -s -T - 'http://localhost:28880/_test/_test.tgz'


In [9]:
contents = os.popen("curl http://localhost:28880/_test/_test.tgz | tar -ztvf -").read()
print contents
assert "002.s" in contents

-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:34 000.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:34 001.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:34 002.s



In [10]:
result = list(htpar.tarrecords("http://localhost:28880/_test/_test.tgz"))
result

# curl -s 'http://localhost:28880/_test/_test.tgz'


[{'__key__': '000', 's': '000'},
 {'__key__': '001', 's': '001'},
 {'__key__': '002', 's': '002'}]

In [11]:
assert len(result) == 3
assert result[0]["s"] == "000"
assert result[2]["__key__"] == "002"

In [12]:
result1 = list(htpar.tarshards("http://localhost:28880/_test/_test.tgz"))
assert result == result1

# curl -s 'http://localhost:28880/_test/_test.tgz'


In [13]:
with htpar.TarShards("http://localhost:28880/_test/_test-@0004.tgz") as stream:
    for i in range(100):
        s = "%03d" % i
        stream.write(s, dict(s=s))

# curl -s -T - 'http://localhost:28880/_test/_test-0000.tgz'
# curl -s -T - 'http://localhost:28880/_test/_test-0001.tgz'
# curl -s -T - 'http://localhost:28880/_test/_test-0002.tgz'
# curl -s -T - 'http://localhost:28880/_test/_test-0003.tgz'
[opened 4 output streams]


In [14]:
contents = os.popen("curl http://localhost:28880/_test/_test-0001.tgz | tar -ztvf -").read()
print contents
assert "030.s" in contents

-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 007.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 009.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 013.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 014.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 017.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 018.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 026.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 028.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 030.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 036.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 038.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 039.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 040.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 042.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 044.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 047.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 052.s
-rw-rw-rw- bigdata/bigdata   3 2017-12-28 09:35 058.s
-rw-rw-rw- bigdata/bigdata  

In [15]:
result = list(htpar.tarshards("http://localhost:28880/_test/_test-@0004.tgz"))
result[:5]

# curl -s 'http://localhost:28880/_test/_test-0000.tgz'
# curl -s 'http://localhost:28880/_test/_test-0001.tgz'
# curl -s 'http://localhost:28880/_test/_test-0002.tgz'
# curl -s 'http://localhost:28880/_test/_test-0003.tgz'


[{'__key__': '001', 's': '001'},
 {'__key__': '004', 's': '004'},
 {'__key__': '005', 's': '005'},
 {'__key__': '010', 's': '010'},
 {'__key__': '011', 's': '011'}]

In [16]:
assert len(result) == 100

In [17]:
assert set(x["s"] for x in result) == \
       set("%03d" % i for i in range(100))

# Shut Down Server

In [18]:
!cd NGINX && docker-compose -f docker-test.yml down || true

/work/htpar/NGINX
Stopping nginx_htpar-test_1 ... 
Removing nginx_htpar-test_1 ... 
Removing network nginx_default32mdone
ERROR: network nginx_default id 1c8ba7a55ca48d24f80f917a4354e1a817a2b947f28aa476b62de10b68608c56 has active endpoints
